In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from pandas.io.json import json_normalize

In [3]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]


In [4]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

,name,population
0,Dade,12345
1,Broward,40000
2,Palm Beach,60000
3,Summit,1234
4,Cuyahoga,1337


In [5]:
 "# further populate tables created from nested element\n",
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

,name,population,state,shortname,info.governor
0,Dade,12345,Florida,FL,Rick Scott
1,Broward,40000,Florida,FL,Rick Scott
2,Palm Beach,60000,Florida,FL,Rick Scott
3,Summit,1234,Ohio,OH,John Kasich
4,Cuyahoga,1337,Ohio,OH,John Kasich


In [ ]:
#JSON example, with file
#demonstrates reading in a json file as a string and as a table
#uses small sample file containing data about projects funded by the World Bank
#data source: http://jsonstudio.com/resources/

In [ ]:
#1 - Find the 10 countries with most projects

In [37]:
# Load data 
path='C:/Users/sakir/OneDrive/Desktop/json/data/world_bank_projects.json'
with open(path) as file:
    data = json.load(file)

In [38]:
# Extract columns from JSON file
df = json_normalize(data, record_path=[['mjtheme_namecode']],
              meta= ['countryname',
                    'project_name'],
              errors = 'ignore')

In [45]:
# There are Duplicate values in columns Countryname and Projectname
df_1 = df[['project_name','countryname']].drop_duplicates()\
.reset_index(drop=True)

In [46]:
top10 = df_1.groupby('countryname')['project_name'].count()\
.sort_values(ascending=False)[:10]

In [47]:
top_tens.head(10)

countryname
People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Nepal                              12
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Africa                             11
Republic of Mozambique             11
Name: project_name, dtype: int64

In [48]:
df.head()

,code,name,countryname,project_name
0,8,Human development,Federal Democratic Republic of Ethiopia,Ethiopia General Education Quality Improvement...
1,11,,Federal Democratic Republic of Ethiopia,Ethiopia General Education Quality Improvement...
2,1,Economic management,Republic of Tunisia,TN: DTF Social Protection Reforms Support
3,6,Social protection and risk management,Republic of Tunisia,TN: DTF Social Protection Reforms Support
4,5,Trade and integration,Tuvalu,Tuvalu Aviation Investment Project - Additiona...


In [ ]:
#2. Find the top 10 major project themes

In [50]:
# Required columns
df_2 = df.loc[:,['code', 'name']]

# Fill empty cells in column 'name'   
df_2.name.replace('', 'Missing_Values', inplace=True)

In [54]:
df_2.name.value_counts()[:10]

Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
Missing_Values                                  122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Name: name, dtype: int64

In [ ]:
#3.Some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [55]:
# Replace Missing Values with NaNs 
df_2.name.replace( to_replace='Missing_Values',value=np.nan, inplace=True)

# Create a dictionary having codes for Corresponding project themes
codes = df_2.code.unique()
mapping = {code : df_2.dropna(subset=['name'])\
           .name.loc[df_2.dropna(subset=['name']).code == code]\
           .values[0] for code in codes}

In [59]:
mapping

{'8': 'Human development',
 '11': 'Environment and natural resources management',
 '1': 'Economic management',
 '6': 'Social protection and risk management',
 '5': 'Trade and integration',
 '2': 'Public sector governance',
 '7': 'Social dev/gender/inclusion',
 '4': 'Financial and private sector development',
 '10': 'Rural development',
 '9': 'Urban development',
 '3': 'Rule of law'}

In [61]:
# Use 'mapping' dictionary to map codes to fill empty project names
df_2['New_Name'] = df_2['code'].map(mapping)
df_2.loc[df_2.name.isnull()].head(10)

,code,name,New_Name
1,11,NaN,Environment and natural resources management
13,6,NaN,Social protection and risk management
17,8,NaN,Human development
19,7,NaN,Social dev/gender/inclusion
24,2,NaN,Public sector governance
29,4,NaN,Financial and private sector development
40,2,NaN,Public sector governance
42,7,NaN,Social dev/gender/inclusion
80,4,NaN,Financial and private sector development
98,11,NaN,Environment and natural resources management
